Created on Thu Oct 26 10:28:08 2023

@author: Santiago D'hers

Use:

- This script will create the geolabels and calculate the distance traveled

Requirements:

- The position.csv files processed by 1-Manage_H5.py

In [12]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import plotly.graph_objects as go

import random

In [ ]:
class Point:
    def __init__(self, df, table):

        x = df[table + '_x']
        y = df[table + '_y']

        self.positions = np.dstack((x, y))[0]

    @staticmethod
    def dist(p1, p2):
        return np.linalg.norm(p1.positions - p2.positions, axis=1)

class Vector:
    def __init__(self, p1, p2, normalize=True):

        self.positions = p2.positions - p1.positions

        self.norm = np.linalg.norm(self.positions, axis=1)

        if normalize:
            self.positions = self.positions / np.repeat(np.expand_dims(self.norm,axis=1), 2, axis=1)

    @staticmethod
    def angle(v1, v2):
        
        length = len(v1.positions)
        angle = np.zeros(length)

        for i in range(length):
            angle[i] = np.rad2deg(np.arccos(np.dot(v1.positions[i], v2.positions[i])))

        return angle

In [2]:
def find_files(path_name, exp_name, group, folder):
    """
    This function finds the files that we want to use and lists their path

    Args:
        path_name (str): _description_
        exp_name (str): _description_
        group (str): _description_
        folder (str): _description_

    Returns:
        list: _description_
    """    
    
    files_path = os.path.join(path_name, exp_name, group, folder)
    files = os.listdir(files_path)
    wanted_files = []
    
    for file in files:
        if f"_{folder}.csv" in file:
            wanted_files.append(os.path.join(files_path, file))
            
    wanted_files = sorted(wanted_files)
    
    return wanted_files

In [3]:
# State your path:
path = r'C:\Users\dhers\OneDrive - UBA\workshop'
experiment = r'2024-09_Tg-1y'

# Hab_position = find_files(path, experiment, "Hab", "position")
# TR_position = find_files(path, experiment, "TR", "position")
TS_position = find_files(path, experiment, "TS", "position")

all_position = TS_position # + TR_position + Hab_position

In [4]:
video = random.randint(1, len(TS_position)) # Select the number of the video you want to use
example = TS_position[video - 1]

In [5]:
def calculate_distances(df, point1, point2):
    x1, y1 = df[point1 + '_x'], df[point1 + '_y']
    x2, y2 = df[point2 + '_x'], df[point2 + '_y']
    
    distances = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distances

In [6]:
def calculate_angles(df, point1, point2, point3):
    # Extract coordinates
    x1, y1 = df[point1 + '_x'], df[point1 + '_y']
    x2, y2 = df[point2 + '_x'], df[point2 + '_y']
    x3, y3 = df[point3 + '_x'], df[point3 + '_y']
    
    # Calculate vectors
    v1 = np.vstack((x1 - x2, y1 - y2)).T
    v2 = np.vstack((x3 - x2, y3 - y2)).T
    
    # Calculate dot product and magnitudes
    dot_p = np.einsum('ij,ij->i', v1, v2)
    m1 = np.linalg.norm(v1, axis=1)
    m2 = np.linalg.norm(v2, axis=1)
    
    # Calculate angles in radians and then convert to degrees
    cos_theta = dot_p / (m1 * m2)
    angles = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))  # Clip to handle numerical errors

    return angles

In [39]:
def plot_position(file, maxDistance = 2.5, maxAngle = 45):
    
    # Read the .csv
    df = pd.read_csv(file)
    
    # Extract positions of both objects and bodyparts
    obj1 = Point(df, 'obj_1')
    obj2 = Point(df, 'obj_2')
    nose = Point(df, 'nose')
    head = Point(df, 'head')
    
    # Find distance from the nose to each object
    dist1 = Point.dist(nose, obj1)
    dist2 = Point.dist(nose, obj2)
    
    # Compute normalized head-nose and head-object vectors
    head_nose = Vector(head, nose, normalize = True)
    head_obj1 = Vector(head, obj1, normalize = True)
    head_obj2 = Vector(head, obj2, normalize = True)
    
    # Find the angles between the head-nose and head-object vectors
    angle1 = Vector.angle(head_nose, head_obj1) # deg
    angle2 = Vector.angle(head_nose, head_obj2) # deg
    
    # Im asking the nose be closer to the aimed object to filter distant sighting
    towards1 = nose.positions[(angle1 < maxAngle)]
    towards2 = nose.positions[(angle2 < maxAngle)]

    # Create traces for the nose positions
    nose_trace = go.Scatter(
    x=nose.positions[:, 0],
    y=nose.positions[:, 1],
    mode='markers',
    marker=dict(color='grey', opacity=0.2),
    name='Nose Positions'
)

# Create traces for the filtered points
    towards1_trace = go.Scatter(
    x=towards1[:, 0],
    y=towards1[:, 1],
    mode='markers',
    marker=dict(color='brown', opacity=0.4),
    name='Oriented towards 1'
)

    towards2_trace = go.Scatter(
    x=towards2[:, 0],
    y=towards2[:, 1],
    mode='markers',
    marker=dict(color='teal', opacity=0.2),
    name='Oriented towards 2'
)

# Create traces for the objects
    obj1_trace = go.Scatter(
    x=[obj1.positions[0][0]],
    y=[obj1.positions[0][1]],
    mode='markers',
    marker=dict(symbol='square', size=18, color='blue', line=dict(color='blue', width=2)),
    name='Object 1'
)

    obj2_trace = go.Scatter(
    x=[obj2.positions[0][0]],
    y=[obj2.positions[0][1]],
    mode='markers',
    marker=dict(symbol='circle', size=20, color='red', line=dict(color='darkred', width=2)),
    name='Object 2'
)

# Create circles around the objects
    circle1 = go.Scatter(
    x=obj1.positions[0][0] + maxDistance * np.cos(np.linspace(0, 2 * np.pi, 100)),
    y=obj1.positions[0][1] + maxDistance * np.sin(np.linspace(0, 2 * np.pi, 100)),
    mode='lines',
    line=dict(color='orange', dash='dash'),
    fill='toself',
    fillcolor='rgba(255, 165, 0, 0.3)',
    name='Object 1 Radius'
)

    circle2 = go.Scatter(
    x=obj2.positions[0][0] + maxDistance * np.cos(np.linspace(0, 2 * np.pi, 100)),
    y=obj2.positions[0][1] + maxDistance * np.sin(np.linspace(0, 2 * np.pi, 100)),
    mode='lines',
    line=dict(color='orange', dash='dash'),
    fill='toself',
    fillcolor='rgba(255, 165, 0, 0.3)',
    name='Object 2 Radius'
)

    # Combine all traces
    data = [nose_trace, towards1_trace, towards2_trace, obj1_trace, obj2_trace, circle1, circle2]

    # Extract the filename without extension
    filename = os.path.splitext(os.path.basename(file))[0]

    # Create layout
    layout = go.Layout(
    # title=f'Analysis of {filename}',
    xaxis=dict(title='Horizontal position (cm)'),
    yaxis=dict(title='Vertical position (cm)'),
    legend=dict(yanchor="bottom",
                y=1,
                xanchor="center",
                x=0.5,
                orientation="h"),
    showlegend=True,
    width=620,
    height=480,
)

    # Create figure
    fig = go.Figure(data=data, layout=layout)

    # Show plot
    fig.show()

In [ ]:
plot_position(example, maxDistance = 2.5, maxAngle = 45)

In [41]:
def create_geolabels(files, maxDistance = 2.5, maxAngle = 45, nan_to_0 = False):
    
    for file in files:
        
        # Determine the output file path
        input_dir, input_filename = os.path.split(file)
        parent_dir = os.path.dirname(input_dir)
        
        # Read the file
        position = pd.read_csv(file)
        
        # Remove the rows where the mouse is still not in the video, excluding the first 4 columns (the object)
        original_rows = position.shape[0]
        position.dropna(subset = position.columns[4:], inplace=True)
        position.reset_index(drop=True, inplace=True)
        rows_removed = original_rows - position.shape[0]
    
        # Extract positions of both objects and bodyparts
        obj1 = Point(position, 'obj_1')
        obj2 = Point(position, 'obj_2')
        nose = Point(position, 'nose')
        head = Point(position, 'head')
    
        # Calculate distances
        dist1 = Point.dist(nose, obj1)
        dist2 = Point.dist(nose, obj2)
    
        # Calculate angles
        head_nose = Vector(head, nose, normalize=True)
        head_obj1 = Vector(head, obj1, normalize=True)
        head_obj2 = Vector(head, obj2, normalize=True)
    
        angle1 = Vector.angle(head_nose, head_obj1)
        angle2 = Vector.angle(head_nose, head_obj2)
        
        if "Hab" not in file:
            
            # Create the geolabels dataframe
            geolabels = pd.DataFrame(np.zeros((position.shape[0], 2)), columns=["Left", "Right"]) 
            
            for i in range(position.shape[0]):
                
                # Check if mouse is exploring object 1
                if dist1[i] < maxDistance and angle1[i] < maxAngle:
                    geolabels.loc[i, "Left"] = 1
        
                # Check if mouse is exploring object 2
                elif dist2[i] < maxDistance and angle2[i] < maxAngle:
                    geolabels.loc[i, "Right"] = 1

            geolabels['Left'] = geolabels['Left'].astype(int)
            geolabels['Right'] = geolabels['Right'].astype(int)
            
            # Add rows filled with zeros at the beginning of geolabels
            zeros_rows = pd.DataFrame(np.nan, index=np.arange(rows_removed), columns=geolabels.columns)
            geolabels = pd.concat([zeros_rows, geolabels]).reset_index(drop=True)
            
            # Insert a new column with the frame number at the beginning of the DataFrame
            geolabels.insert(0, "Frame", geolabels.index + 1)
            
            if nan_to_0:
                # Fill any remaining nan with 0
                geolabels.fillna(0, inplace=True)
            
            # Create a filename for the output CSV file
            output_filename_geolabels = input_filename.replace('_position.csv', '_geolabels.csv')
            output_folder_geolabels = os.path.join(parent_dir + '/geolabels')
            os.makedirs(output_folder_geolabels, exist_ok = True)
            output_path_geolabels = os.path.join(output_folder_geolabels, output_filename_geolabels)
            geolabels.to_csv(output_path_geolabels, index=False)
            
            print(f"Saved geolabels to {output_filename_geolabels}")
        
        # Create the distances dataframe
        distances = pd.DataFrame(np.zeros((position.shape[0], 2)), columns=["nose_dist", "body_dist"])
        
        # Calculate the Euclidean distance between consecutive nose positions
        distances['nose_dist'] = (((position['nose_x'].diff())**2 + (position['nose_y'].diff())**2)**0.5) / 100
        distances['body_dist'] = (((position['body_x'].diff())**2 + (position['body_y'].diff())**2)**0.5) / 100
        
        # Add rows filled with zeros at the beginning of distances
        zeros_rows = pd.DataFrame(np.nan, index=np.arange(rows_removed), columns=distances.columns)
        distances = pd.concat([zeros_rows, distances]).reset_index(drop=True)
        
        # Insert a new column with the frame number at the beginning of the DataFrame
        distances.insert(0, "Frame", distances.index + 1)

        
        if nan_to_0:
            # Fill any remaining nan with 0
            distances.fillna(0, inplace=True)
        
        output_filename_distances = input_filename.replace('_position.csv', '_distances.csv')
        output_folder_distances = os.path.join(parent_dir + '/distances')
        os.makedirs(output_folder_distances, exist_ok = True)
        output_path_distances = os.path.join(output_folder_distances, output_filename_distances)
        distances.to_csv(output_path_distances, index=False)
            
        print(f"Saved distances to {output_filename_distances}")

In [42]:
create_geolabels(all_position)

Saved geolabels to 2024-09_Tg-1y_TS_R01_C03-i_A_L_geolabels.csv
Saved distances to 2024-09_Tg-1y_TS_R01_C03-i_A_L_distances.csv
Saved geolabels to 2024-09_Tg-1y_TS_R02_C03-d_A_R_geolabels.csv
Saved distances to 2024-09_Tg-1y_TS_R02_C03-d_A_R_distances.csv
Saved geolabels to 2024-09_Tg-1y_TS_R03_C03-a_B_R_geolabels.csv
Saved distances to 2024-09_Tg-1y_TS_R03_C03-a_B_R_distances.csv
Saved geolabels to 2024-09_Tg-1y_TS_R04_C04-i_A_L_geolabels.csv
Saved distances to 2024-09_Tg-1y_TS_R04_C04-i_A_L_distances.csv
Saved geolabels to 2024-09_Tg-1y_TS_R05_C04-d_A_R_geolabels.csv
Saved distances to 2024-09_Tg-1y_TS_R05_C04-d_A_R_distances.csv
Saved geolabels to 2024-09_Tg-1y_TS_R07_C05-i_B_L_geolabels.csv
Saved distances to 2024-09_Tg-1y_TS_R07_C05-i_B_L_distances.csv
Saved geolabels to 2024-09_Tg-1y_TS_R08_C05-a_B_R_geolabels.csv
Saved distances to 2024-09_Tg-1y_TS_R08_C05-a_B_R_distances.csv
Saved geolabels to 2024-09_Tg-1y_TS_R09_C06-i_A_L_geolabels.csv
Saved distances to 2024-09_Tg-1y_TS_R09_